In [1]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

from keras_image_helper import create_preprocessor

2024-12-31 01:22:11.267890: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-31 01:22:11.271195: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-31 01:22:11.280203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735608131.294771   12057 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735608131.299123   12057 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-31 01:22:11.315283: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [3]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [4]:
url = 'https://github.com/ranga4all1/geo-image-classification/blob/main/images/glacier.jpg?raw=true'
X = preprocessor.from_url(url)

In [5]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [6]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'saved-geo-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_layer_1'].CopyFrom(np_to_protobuf(X))

In [7]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [8]:
preds = pb_response.outputs['output_0'].float_val

In [9]:
classes = [
    'buildings',
    'forest',
    'glacier',
    'mountain',
    'sea',
    'street'
]

In [10]:
dict(zip(classes, preds))

{'buildings': -0.7047825455665588,
 'forest': -1.4779146909713745,
 'glacier': 2.144595146179199,
 'mountain': 1.1838254928588867,
 'sea': -0.41370701789855957,
 'street': -0.41950517892837524}